# Promptfoo: custom code graders

**Note: このレッスンは関連コードファイルを含むフォルダ内にあります。実際に手元で実行しながら進めたい場合は、フォルダ全体をダウンロードしてください。**

これまで、promptfoo の組み込み採点（grader）である `exact-match` や `contains-all` のようなものを使ってきました。これらは便利ですが、promptfoo では **より特定の採点タスク向けに、独自の採点ロジック（カスタムグレーダ）** を書くこともできます。

それを示すために、次のシンプルなプロンプトテンプレートを使います：

> Write a short paragraph about {{topic}}. Make sure you mention {{topic}} exactly {{count}} times, no more or fewer.

`{{topic}}` と `{{count}}` に例えば `"tweezers"` と `7` を入れると、次のようなプロンプトになります：

> Write a short paragraph about tweezers. Make sure you mention tweezers exactly 7 times, no more or fewer.

これを採点するには、モデル出力に "tweezers" がちょうど7回出現することをチェックするカスタムロジックが必要です。

例えば次のプロンプト：

> Write a short paragraph about sheep. Make sure you mention sheep exactly 3 times, no more or fewer.

では、モデル出力に "sheep" がちょうど3回含まれていることを確認する採点ロジックが必要です。

---


## Initializing promptfoo

いつも通り、まずは次のコマンドで promptfoo を初期化します：

```bash
npx promptfoo@latest init
```

これまでと同様に `promptfooconfig.yaml` が作成されます。既存の内容は削除して構いません。

次にプロバイダを設定します。`promptfooconfig.yaml` に次を追加してください：

```yaml
description: Count mentions

providers:
  - anthropic:messages:claude-3-haiku-20240307
  - anthropic:messages:claude-3-5-sonnet-20240620
```

これにより、Claude 3 Haiku と Claude 3.5 Sonnet の両方で評価を回し、このタスクでの性能を比較できます。

`ANTHROPIC_API_KEY` 環境変数が設定されていることを確認してください。ターミナルで次のように設定できます：

```bash
export ANTHROPIC_API_KEY=your_api_key_here
```

---


## Preparing our prompts

これまでは Python ファイルに関数としてプロンプトを書く方法を推奨してきましたが、promptfoo には他にもいくつかの指定方法があります。最もシンプルなのは、YAMLファイルにプロンプト本文を直接書く方法です。

ここではこのインライン方式を試します。`promptfooconfig.yaml` を次のように更新してください：

```yaml
description: Count mentions
prompts:
  - >-
    {{topic}}について短い段落を書いてください。{{topic}}をちょうど{{count}}回、それ以上でもそれ以下でもなく言及してください。出力では小文字のみを使用してください。
providers:
  - anthropic:messages:claude-3-haiku-20240307
  - anthropic:messages:claude-3-5-sonnet-20240620
```

`prompts` フィールドに、テキストプロンプトを直接書いている点に注目してください。また `{{topic}}` と `{{count}}` のように二重波括弧を使っています。これは Nunjucks のテンプレート構文で、この後重要になります。

---


## Writing our test cases

以前のレッスンでは CSV にテストケースと採点ロジックを書きました。繰り返しになりますが、promptfoo は非常に柔軟で、テストの指定方法も複数あります。

ここでは、テストケースを YAML 設定ファイル内に直接書きます。`promptfooconfig.yaml` を次のようにしてください：

```yaml
description: Count mentions
prompts:
  - >-
    {{topic}}について短い段落を書いてください。{{topic}}をちょうど{{count}}回、それ以上でもそれ以下でもなく言及してください。出力では小文字のみを使用してください。
providers:
  - anthropic:messages:claude-3-haiku-20240307
  - anthropic:messages:claude-3-5-sonnet-20240620
tests:
  - vars:
      topic: sheep
      count: 3
  - vars:
      topic: fowl
      count: 2
  - vars:
      topic: gallows
      count: 4
  - vars:
      topic: tweezers
      count: 7
  - vars:
      topic: jeans
      count: 6
```

末尾で5つのテストケースを定義しており、それぞれ `topic` と `count` が異なります。promptfoo は各テストを自動実行し、テンプレート内の `{{topic}}` と `{{count}}` を置換します。

まだ採点ロジックはありませんが、まずは変数が正しく埋め込まれているか確認するために評価を実行できます。

評価はこれまでと同じコマンドです：

```bash
npx promptfoo@latest eval
```


出力例は次のとおりです：

![initial_eval_output.png](../images/initial_eval_output.png)

1行だけ拡大すると、モデル出力は概ね良さそうに見えます。この例では `{{topic}}` が "sheep" になっており、出力も羊についての段落になっています。

![single_row-2.png](../images/single_row.png)

あとは、出力がトピックを指定回数ちょうど含むかをチェックするカスタム採点ロジックを実装するだけです。


---

## Adding a custom grader function

promptfoo では Python で独自の採点関数を書けます。この例では、モデル出力に指定トピックが指定回数ちょうど出現することを保証する関数を作ります。

`count.py` という新しい Python ファイルを作り、次の関数を追加します：

```py
import re

def get_assert(output, context):
    topic = context["vars"]["topic"]
    goal_count = int(context["vars"]["count"])
    pattern = fr'(^|\s){re.escape(topic)}'

    actual_count = len(re.findall(pattern, output.lower()))

    pass_result = goal_count == actual_count

    result = {
        "pass": pass_result,
        "score": 1 if pass_result else 0,
        "reason": f"Expected {topic} to appear {goal_count} times. Actual: {actual_count}",
    }
    return result
```

上のコードの動きを説明します。promptfoo はファイル内の `get_assert` という関数を自動的に探し、次の2引数を渡します：

- あるモデルの出力（output）
- その出力を生成した変数やプロンプトを含む `context` 辞書

promptfoo は、関数の戻り値として次のいずれかを受け付けます：
- bool（合否）
- float（スコア）
- GradingResult 辞書

ここでは GradingResult 辞書を返します。これには次のプロパティが必要です：

- `pass_`：boolean
- `score`：float
- `reason`：説明文（string）

この関数では、`context` から topic と count を取り出し、正規表現でトピック出現回数を数え、結果を `result` として返しています。


グレーダを定義できたので、次は promptfoo にそれを使うよう伝えます。`promptfooconfig.yaml` を次のように更新してください：

```yaml
description: Count mentions
prompts:
  - >-
    {{topic}}について短い段落を書いてください。{{topic}}をちょうど{{count}}回、それ以上でもそれ以下でもなく言及してください。出力では小文字のみを使用してください。
providers:
  - anthropic:messages:claude-3-haiku-20240307
  - anthropic:messages:claude-3-5-sonnet-20240620
defaultTest:
  assert:
    - type: python
      value: file://count.py
tests:
  - vars:
      topic: sheep
      count: 3
  - vars:
      topic: fowl
      count: 2
  - vars:
      topic: gallows
      count: 4
  - vars:
      topic: tweezers
      count: 7
  - vars:
      topic: jeans
      count: 6
```

`defaultTest` は、各テストを実行する際に `count.py` の Python グレーダを使うことを指定しています。

---


## Running the evaluation

評価を実行するには、これまでと同じコマンドを使います：

```bash
npx promptfoo@latest eval
```


評価を実行すると、次のような結果になります：

![final_eval.png](../images/final_eval.png)

Webインターフェースを開くには次を実行します：

```bash
npx promptfoo@latest view
```

![final_view.png](../images/final_view.png)


このタスクでは、Claude 3.5 が 100% で、Claude 3 Haiku は 20% でした。結果を確認するには、虫眼鏡アイコンをクリックして入力プロンプトと対応する出力を表示します。

Claude 3 Haiku の誤った出力例：

![tweezers_haiku_closeup.png](../images/tweezers_haiku_closeup.png)

Claude 3.5 Sonnet の正しい出力例：

![tweezers_sonnet_closeup.png](../images/tweezers_sonnet_closeup.png)


この評価自体は少しふざけた題材ですが、目的は「カスタムの Python 採点ロジックを定義する手順」を示すことです。promptfoo の組み込みアサーションとカスタムグレーダ関数を組み合わせれば、ほぼ任意のコード採点評価を作れます。

次のレッスンでは、promptfoo におけるモデル採点（model-graded）の評価を学びます。
